This worksheet automates the creation of Cut 1 table where the initial set of filters and enrichment is done

In [22]:
from google.cloud import bigquery

In [24]:
client = bigquery.Client()

In [25]:
# This is the version 20240215

query = """
CREATE OR REPLACE TABLE analysis.INSERT_TABLE_NAME AS
WITH cut_1 AS (
  SELECT DISTINCT ein
  FROM original.eo_extract_990_2022
  WHERE subseccd = '03'
    AND nonpfrea in (1, 7, 9)
    AND totfuncexpns BETWEEN 250000 AND 18000000
    AND totrevenue BETWEEN 250000 AND 24000000
    AND ein IS NOT NULL
),
additional_orgs AS (
  SELECT DISTINCT CAST(ein AS STRING) AS ein
  FROM reference.additional_organizations_native
),
additional_orgs_union AS (
  SELECT *
  FROM cut_1

  UNION DISTINCT

  SELECT *
  FROM additional_orgs
),
acs_data AS (
  SELECT DISTINCT e.ein, a.*
  FROM reference.ein_county_2022 AS e
  LEFT JOIN reference.acs_data_final_native AS a ON RIGHT(geo_id, 5) = e.county
),
coords AS (
  SELECT DISTINCT a.*, c.latitude, c.longitude
  FROM acs_data AS a
  LEFT JOIN reference.ein_lat_lon_2022 AS c ON a.ein = c.ein
)
SELECT
  i.*, a.* EXCEPT (ein)
FROM additional_orgs_union AS c
LEFT JOIN original.irs_990_latest AS i ON c.ein = i.ein
LEFT JOIN coords AS a ON a.ein = c.ein
WHERE i.ein IS NOT NULL AND address IS NOT NULL
AND EXTRACT(YEAR FROM tax_period_end_date) IN (2021, 2022)
-- for each ein that made the first cut, only get their latest tax period data
QUALIFY ROW_NUMBER() OVER (PARTITION BY ein ORDER BY tax_period_end_date DESC) = 1
"""

In [26]:
query_job = client.query(query)

In [27]:
query_job.result()